In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from urllib import quote_plus as urlquote

pd.set_option('max_columns',1000)
pd.set_option('max_row',300)
pd.set_option('display.float_format', lambda x: '%.5f' % x)


# 开思平台生产库
password = "Casstime@0719"
engine = create_engine('mysql+pymysql://ximu:%s@cassec-demo.mysql.rds.aliyuncs.com:3306/ec_admin'%urlquote(password))

# 开思金融数据库
password2 = "Casstime@com1112"
engine2 = create_engine('mysql+pymysql://reader:%s@172.21.21.78:3306/kscredit'%urlquote(password2))

def get_sql(sql,engine):
    df = pd.read_sql(sql=sql,con = engine)
    print df.shape
    print df.columns
    return df

In [34]:
sql2 = '''
SELECT distinct t1.product_store_id,
DATE_FORMAT(t1.created_stamp,'%%Y-%%m') as yymm,
sum(t2.ACTUAL_CURRENCY_AMOUNT) AS amt
FROM ofbiz.order_header t1
      JOIN ofbiz.order_payment t2
      ON t1.ORDER_ID = t2.ORDER_ID
      JOIN ec_admin.PARTY_GROUP_GEO t3
      ON t1.company_id = t3.party_id
      JOIN ec_admin.party_group t4
      ON t1.product_store_id = t4.code
      JOIN ec_admin.party t5
      ON t4.id = t5.id
      AND t5.party_category = 'supplier'
      JOIN (SELECT DISTINCT product_store_id,ORDER_ID 
            FROM (
            SELECT DISTINCT t2.product_store_id,t2.ORDER_ID,
            '白条' as loan_type
            FROM ofbiz.cass_credit_record t1
            JOIN ofbiz.order_header t2
            ON t1.ORDER_ID = t2.ORDER_ID
            WHERE credit_status NOT LIKE "REFUNDED" and CREDIT_AMOUNT_ACTUAL > 0 
            AND  t2.company_id not in ("ZDQL7blRgcYkehvzLJx","qV80ZU26BkEvQCjNeyG","PQVDbTECtqtL9RU3LM1","ojqQZRmNiaXqwaxQpnL",
            "lKe1gf5KsQF2Qvr7lqa","k48dr6BDLfB1kk65HZl","4UT4HuPaG2MZPa440GU")
            and t2.product_store_id not IN ("XMHYSX01","TSJWL001","GZCDSX01","GZBSSX01","GZLRH001")
            and t2.created_stamp >= "2019-01-01"
            AND t2.status_id IN ("ORDER_APPROVED","ORDER_COMPLETED") 
            union 
            SELECT DISTINCT t2.product_store_id,t2.ORDER_ID,
            '挂账' as loan_type
            FROM ofbiz.payment_bill_header t1
            JOIN ofbiz.order_header t2
            ON t1.deal_id = t2.ORDER_ID
            LEFT JOIN ofbiz.p_b_group_header_rel t3
            ON t1.p_b_h_id = t3.P_B_H_ID
            JOIN ofbiz.payment_bill_group t4
            ON t3.BILL_GROUP_ID = t4.BILL_GROUP_ID AND  t4.type LIKE "COMPANY_ACCOUNT" AND t4.deal_amount > 0
            LEFT JOIN ofbiz.payment t5
            ON t4.PAYMENT_ID = t5.payment_id 
            LEFT JOIN (SELECT payment_group_id,MAX(created_stamp) AS repay_time FROM ofbiz.payment_group_member GROUP BY payment_group_id) t8
            ON t5.payment_group_id = t8.payment_group_id
            LEFT JOIN ofbiz.p_b_group_classify_rel t6
            ON t4.bill_group_id = t6.bill_group_id 
            JOIN ofbiz.payment_bill_classify t7
            ON t6.classify_id = t7.classify_id AND t7.classify_type = "BUYERBILL"
            WHERE t2.company_id NOT IN ("ZDQL7blRgcYkehvzLJx","qV80ZU26BkEvQCjNeyG","PQVDbTECtqtL9RU3LM1","ojqQZRmNiaXqwaxQpnL",
            "lKe1gf5KsQF2Qvr7lqa","k48dr6BDLfB1kk65HZl","4UT4HuPaG2MZPa440GU")
            AND t2.product_store_id NOT IN ("XMHYSX01","TSJWL001","GZCDSX01","GZBSSX01","GZLRH001")
            AND T2.CREATED_STAMP >= "2019-01-01"
            )a
           )od on t1.order_id = od.order_id and t1.product_store_id = od.product_store_id
      WHERE t2.actual_currency_amount >= 0.01
      AND t2.actual_currency_amount<=6000000
      AND t1.company_id NOT IN ("ZDQL7blRgcYkehvzLJx","qV80ZU26BkEvQCjNeyG","PQVDbTECtqtL9RU3LM1","ojqQZRmNiaXqwaxQpnL",
      "lKe1gf5KsQF2Qvr7lqa","k48dr6BDLfB1kk65HZl","4UT4HuPaG2MZPa440GU")
      AND t1.product_store_id NOT IN ("XMHYSX01","TSJWL001","GZCDSX01","GZBSSX01","GZLRH001")
      AND t1.created_stamp >= "2019-01-01"
      AND t1.status_id IN ("ORDER_APPROVED","ORDER_COMPLETED")
group by t1.product_store_id,DATE_FORMAT(t1.created_stamp,'%%Y-%%m')
'''

In [37]:
df2 = get_sql(sql2,engine)
df2.to_excel(u"D:/casstime_ana/11_xiaoqian/test.xlsx")

(9771, 3)
Index([u'product_store_id', u'yymm', u'amt'], dtype='object')
